# Finetune (training) Hands-on

次の内容の再現確認。 <https://huggingface.co/docs/transformers/ja/training#train-in-native-pytorch> 

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

## Preparing dataset

データセットを読み込む。データセットは [YelpReviewFull](https://huggingface.co/datasets/Yelp/yelp_review_full) でローカルビジネスのレビューとその点数。

In [2]:
dataset = load_dataset("yelp_review_full")
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

トークナイザ(BERTより)を準備

In [3]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

データセットをあらかじめトークナイズしちゃう

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

モデルが受け付けられるように、`text`列を削除し、`label`列を`labels`に変更する。  
データセットの形式をPyTorchテンソルに変更する。

In [5]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

実行時間を短くするために、サブセットで学習&評価する

In [6]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

## Prepare DataLoader and

10個を1バッチとして学習・検証する

In [7]:
from torch.utils.data import DataLoader

batch_size = 10

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=batch_size)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=batch_size)

必要なラベル数を指定してモデルをロードする

In [8]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Optimizer and Learning rate schedular

In [9]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

デフォルトのスケジューラーを作る

In [10]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

GPU(CUDA)が利用できるならば、モデルをGPUにロードする

In [11]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [12]:
for name, param in model.named_parameters():
    print(f"{name:80s} {str(param.shape):30s} {param.dtype} {param.is_cuda}")

bert.embeddings.word_embeddings.weight                                           torch.Size([28996, 768])       torch.float32 True
bert.embeddings.position_embeddings.weight                                       torch.Size([512, 768])         torch.float32 True
bert.embeddings.token_type_embeddings.weight                                     torch.Size([2, 768])           torch.float32 True
bert.embeddings.LayerNorm.weight                                                 torch.Size([768])              torch.float32 True
bert.embeddings.LayerNorm.bias                                                   torch.Size([768])              torch.float32 True
bert.encoder.layer.0.attention.self.query.weight                                 torch.Size([768, 768])         torch.float32 True
bert.encoder.layer.0.attention.self.query.bias                                   torch.Size([768])              torch.float32 True
bert.encoder.layer.0.attention.self.key.weight                                   to

## Training Loop!

メトリックスの計算と表示を行う関数を定義する

In [13]:
import evaluate

def compute_metrics(epoch):
    metric = evaluate.load("accuracy")
    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
    
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])
    print(f"#{epoch}  {metric.compute()}")

学習の流れ
1. データローダーから 1 バッチを読み込む
2. モデルに1バッチを流し込み、結果 `outputs` を得る。おそらく `.loss` には教師データと計算結果の差分が入ってる
3. `loss.backward()` で学習を実施
4. `optimizer` と `lr_schedular` を更新 (学習の収束≒学習率のダンピングを狙ってるんだと思うけど、実際にはどういう効果があるんだろう?)

In [14]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

# 学習前のメトリックスを計算・表示
compute_metrics(0)

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    compute_metrics(epoch+1)

  0%|          | 0/300 [00:00<?, ?it/s]

D:\Python\Python311\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


#0  {'accuracy': 0.165}
#1  {'accuracy': 0.434}
#2  {'accuracy': 0.58}
#3  {'accuracy': 0.596}
